In [1]:
import torch
import torch.nn as nn

import read_trace_code
from importlib import reload
import tools_on_B_graph as Btools

# Test on mymod :

In [2]:
class SubMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential (
            nn.ReLU(),
            nn.Linear(128,512),
        )
    
    def forward(self,p1,p2,p3):
        y1 = self.linear_stack(p1)
        y2 = self.linear_stack(p2)
        y3 = self.linear_stack(p3)
        return y1+y2,y1-y2+y3,y1

class MyMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.op = SubMod()
    def forward(self,x):
        (z1,z2,z3) = self.op(x,-x,p3=torch.ones_like(x))
        return z1 + 2*z3 + z2

mymod = MyMod()

In [3]:
src_mymod = torch.rand((128))
script_mymod = torch.jit.trace_module(mymod, {'forward': (src_mymod,)},check_trace=False)
print("===== MYMOD =====")
print(script_mymod.forward.code)

print("===== SUB CODE : self.op.forward =====")
print(script_mymod.op.code)

print("===== SUB SUB CODE =====")
print(script_mymod.op.linear_stack.forward1.code)

print("===== SUB SUB SUB CODE =====")
print("== to debug : getattr(self.op.linear_stack,\"1\").forward2 ==")
print(getattr(script_mymod.op.linear_stack,"1").forward2.code)

===== MYMOD =====
def forward(self,
    x: Tensor) -> Tensor:
  op = self.op
  input = torch.neg(x)
  input0 = torch.ones_like(x, dtype=6, layout=0, device=torch.device("cpu"), pin_memory=False)
  _0, _1, _2, = (op).forward(x, input, input0, )
  _3 = torch.add(_1, torch.mul(_0, CONSTANTS.c0))
  return torch.add(_3, _2)

===== SUB CODE : self.op.forward =====
def forward(self,
    x: Tensor,
    input: Tensor,
    input0: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
  linear_stack = self.linear_stack
  _0 = (linear_stack).forward(x, )
  _1 = (linear_stack).forward1(input, )
  _2 = (linear_stack).forward2(input0, )
  z1 = torch.add(_0, _1)
  z2 = torch.add(torch.sub(_0, _1), _2)
  return (_0, z1, z2)

===== SUB SUB CODE =====
def forward1(self,
    input: Tensor) -> Tensor:
  _1 = getattr(self, "1")
  _0 = getattr(self, "0")
  _2 = (_1).forward1((_0).forward1(input, ), )
  return _2

===== SUB SUB SUB CODE =====
== to debug : getattr(self.op.linear_stack,"1").forward2 ==
def forward2(self,


In [4]:
reload(read_trace_code)
reload(Btools)
mymod_Bg = read_trace_code.main(mymod,(src_mymod,))
Btools.print_code(mymod_Bg)

Opening : self.forward
Opening : self.op.forward
Opening : self.op.linear_stack.forward
Opening : getattr(self.op.linear_stack,"0").forward
Opening : getattr(self.op.linear_stack,"1").forward
Opening : self.op.linear_stack.forward1
Opening : getattr(self.op.linear_stack,"0").forward1
Opening : getattr(self.op.linear_stack,"1").forward1
Opening : self.op.linear_stack.forward2
Opening : getattr(self.op.linear_stack,"0").forward2
Opening : getattr(self.op.linear_stack,"1").forward2
def main(x):
	__10_fv = torch.relu(x)
	__13_y1 = torch.linear(__10_fv,getattr(self.op.linear_stack,"1").weight,getattr(self.op.linear_stack,"1").bias)
	__2_input = torch.neg(x)
	__18_fv = torch.relu(__2_input)
	__21_y2 = torch.linear(__18_fv,getattr(self.op.linear_stack,"1").weight,getattr(self.op.linear_stack,"1").bias)
	__30_z1 = torch.add(__13_y1,__21_y2)
	__32_fv = torch.sub(__13_y1,__21_y2)
	__4_fv = torch.device(cpu)
	__3_input0 = torch.ones_like(x,dtype = 6,layout = 0,device = __4_fv,pin_memory = False)


In [10]:
""" strange behavior : 
class B_test():
    def __init__(self,req = []):
        self.re = req
        
l = B_test()
l2 = B_test()
l.re.append(5)

print(l.re)
print(l2.re)
"""

' strange behavior : \nclass B_test():\n    def __init__(self,req = []):\n        self.re = req\n        \nl = B_test()\nl2 = B_test()\nl.re.append(5)\n\nprint(l.re)\nprint(l2.re)\n'